<a href="https://colab.research.google.com/github/joaguilar/pong-from-scratch-fixed/blob/main/pg_from_scratch_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Policy Gradient

* http://karpathy.github.io/2016/05/31/rl/
* https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5

## Step 1: Installation and setup for the envionronment



In [1]:
# !apt-get install python-opengl -y 
# !apt install xvfb -y
# !pip install pyvirtualdisplay
# !pip install piglet
# !pip install numpy==1.23.4
# !pip install -U gym==0.26.0
# !pip install -U ale-py==0.8.0

In [2]:
# !wget http://www.atarimania.com/roms/Roms.rar
# !unrar x Roms.rar -y
# !mv "ROMS/Video Olympics - Pong Sports (Paddle) (1977) (Atari, Joe Decuir - Sears) (CX2621 - 99806, 6-99806, 49-75104) ~.bin" "ROMS/pong.bin"
# !ale-import-roms ROMS/

In [3]:
from ale_py import ALEInterface
from ale_py.roms import Pong

ale = ALEInterface()
ale.loadROM(Pong)

In [4]:
# !apt-get update
# !sudo apt-get install xvfb
# !pip install xvfbwrapper

In [5]:
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()

In [6]:
%matplotlib inline
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 144)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    plt.close(anim._fig)
    display(HTML(anim.to_jshtml()))

## Step 2: Setting up the gym environment


In [7]:
import gym
env = gym.make('ALE/Pong-v5', render_mode='rgb_array')

In [8]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [9]:
env.action_space

Discrete(6)

In [10]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [11]:
# Run a demo of the environment
observation = env.reset()
cumulated_reward = 0

frames = []
for t in range(1000):
#     print(observation)play_game
    frames.append(env.render())
    # very stupid agent, just makes a random action within the allowd action space
    action = env.action_space.sample()
#     print("Action: {}".format(t+1))    
    observation, reward, done, info, _ = env.step(action)
#     print(reward)
    cumulated_reward += reward
    if done:
        print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
        break
print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))

env.close()

D:\Development\Cenfotec\aa\pong-from-scratch-fixed\pong-from-scratch-fixed\lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode finished after 764 timesteps, accumulated reward = -21.0
Episode finished without success, accumulated reward = -21.0


In [12]:
# display_frames_as_gif(frames)

## Step 3: Policy Gradient from Scratch

In [49]:
import time

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  # I = np.array(I)
  if isinstance(I,tuple):
    return I
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(np.float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):
  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  if isinstance(cur_x,tuple):
    cur_x = prepro(cur_x[0])
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render()) #mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info, _ = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  start = time.time()
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    if isinstance(cur_x,tuple):
      cur_x = prepro(cur_x[0])
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info, _ = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      if (episode_number % 50 == 0):
        ct = time.time()
        minutes = (ct-start)//60
#         print(' start = %f, ct = %f, c-s=%f, m= %f, s = %f' % (start, ct, (ct-start), minutes, (ct-start-(minutes*60))))
        print ('resetting env. episode %f, reward total was %f. running mean: %f in %d minutes %d seconds' % 
               (episode_number, reward_sum, running_reward, minutes, (ct-start-(minutes*60))))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist
    
  end = time.time()
  minutes = (end-start)//60
  print('Elapsed time: %d minutes, %d seconds' % (minutes, (ct-start-(minutes*60))))

    # if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    #   print('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

In [50]:
import numpy as np

# model initialization
H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {} 
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)

# import pickle
# model = pickle.load(open('model.pkl', 'rb')) 

In [51]:
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
# learning_rate = 1e-4 
learning_rate = 1e-3
 
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

In [53]:
%time hist1 = train_model(env, model, total_episodes=6000)

C:\Users\joagu\AppData\Local\Temp\ipykernel_33208\306231196.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return I.astype(np.float).ravel()


resetting env. episode 50.000000, reward total was -20.000000. running mean: -19.571067 in 1 minutes 14 seconds
resetting env. episode 100.000000, reward total was -19.000000. running mean: -19.262206 in 2 minutes 37 seconds
resetting env. episode 150.000000, reward total was -19.000000. running mean: -18.704037 in 4 minutes 7 seconds
resetting env. episode 200.000000, reward total was -17.000000. running mean: -18.245124 in 5 minutes 43 seconds
resetting env. episode 250.000000, reward total was -16.000000. running mean: -17.712860 in 7 minutes 22 seconds
resetting env. episode 300.000000, reward total was -12.000000. running mean: -17.061453 in 9 minutes 12 seconds
resetting env. episode 350.000000, reward total was -14.000000. running mean: -16.626857 in 11 minutes 9 seconds
resetting env. episode 400.000000, reward total was -17.000000. running mean: -15.979905 in 13 minutes 16 seconds
resetting env. episode 450.000000, reward total was -20.000000. running mean: -15.603856 in 15 mi

resetting env. episode 3700.000000, reward total was 11.000000. running mean: 8.935730 in 304 minutes 54 seconds
resetting env. episode 3750.000000, reward total was 11.000000. running mean: 9.333226 in 309 minutes 59 seconds
resetting env. episode 3800.000000, reward total was 11.000000. running mean: 9.638062 in 314 minutes 50 seconds
resetting env. episode 3850.000000, reward total was 14.000000. running mean: 9.444741 in 319 minutes 48 seconds
resetting env. episode 3900.000000, reward total was 16.000000. running mean: 9.439217 in 324 minutes 46 seconds
resetting env. episode 3950.000000, reward total was 16.000000. running mean: 9.484879 in 329 minutes 51 seconds
resetting env. episode 4000.000000, reward total was 4.000000. running mean: 9.266741 in 334 minutes 57 seconds
resetting env. episode 4050.000000, reward total was 11.000000. running mean: 9.444227 in 339 minutes 48 seconds
resetting env. episode 4100.000000, reward total was 7.000000. running mean: 9.015887 in 345 minu

In [54]:
play_game(env, model) 

C:\Users\joagu\AppData\Local\Temp\ipykernel_33208\306231196.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return I.astype(np.float).ravel()


Episode finished without success, accumulated reward = 6.0


In [55]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))